In [1]:
!pip install --upgrade scikit-learn
!pip install --upgrade keras
!pip install --upgrade tensorflow
!pip install --upgrade pandas
!pip install --upgrade numpy
!pip install --upgrade matplotlib
!pip install --upgrade torch


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
  Obtaining dependency information for keras from https://files.pythonhosted.org/packages/b2/e4/30b53d839608d2212b97972a8516ba0c7e776ee1102eaa82624807b944cf/keras-3.0.1-py3-none-any.whl.metadata
  Using cached keras-3.0.1-py3-none-any.whl.metadata (4.8 kB)
Using cached keras-3.0.1-py3-none-any.whl (999 kB)
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-macos 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.1 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
  Obtaining dependency information for

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()


# download the data from the given URLs and store them in dataframes
training_data_df = pd.read_csv('../1_DATA/1_0_PROJECT_DATA/training_data.csv')
unlabelled_test_data_df = pd.read_csv('../1_DATA/1_0_PROJECT_DATA/unlabelled_test_data.csv')

# Exemple de données
texts = training_data_df['sentence'].values.tolist()
labels = training_data_df['difficulty'].values.tolist()



# Création de la matrice TF-IDF
vectorizer = TfidfVectorizer(max_features=14585)  # Ajustez ce nombre pour qu'il corresponde à la taille de vos données
X = vectorizer.fit_transform(texts).toarray()
y = np.array(labels)
y = label_encoder.fit_transform(labels)  # 'labels' est votre liste ou tableau d'étiquettes

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [40]:
# Conversion en tenseurs PyTorch
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)

# Création de DataLoaders
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 32  # Vous pouvez ajuster la taille du batch
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [41]:
# Neural Network Class
class TextClassifier(nn.Module):
    def __init__(self, input_dim):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 128)
        # Adjust the number of output classes if necessary
        self.fc3 = nn.Linear(128, len(np.unique(y)))  

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

model = TextClassifier(14585)  # 5000 correspond au nombre de max_features dans TF-IDF

In [42]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

epochs = 200  # Ajustez le nombre d'époques selon vos besoins
for epoch in range(epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs} - Loss: {loss.item()}")

Epoch 1/200 - Loss: 1.7866032123565674
Epoch 2/200 - Loss: 1.778885841369629
Epoch 3/200 - Loss: 1.7487225532531738
Epoch 4/200 - Loss: 1.6429495811462402
Epoch 5/200 - Loss: 1.5269412994384766
Epoch 6/200 - Loss: 1.4340283870697021
Epoch 7/200 - Loss: 1.2762011289596558
Epoch 8/200 - Loss: 1.2688612937927246
Epoch 9/200 - Loss: 1.1151635646820068
Epoch 10/200 - Loss: 1.0599768161773682
Epoch 11/200 - Loss: 1.007269263267517
Epoch 12/200 - Loss: 0.9394813776016235
Epoch 13/200 - Loss: 0.909389317035675
Epoch 14/200 - Loss: 0.7998769879341125
Epoch 15/200 - Loss: 0.6273455619812012
Epoch 16/200 - Loss: 0.7611042261123657
Epoch 17/200 - Loss: 0.5088938474655151
Epoch 18/200 - Loss: 0.47609075903892517
Epoch 19/200 - Loss: 0.5351194143295288
Epoch 20/200 - Loss: 0.3765825927257538
Epoch 21/200 - Loss: 0.4028696119785309
Epoch 22/200 - Loss: 0.3648737072944641
Epoch 23/200 - Loss: 0.3505692780017853
Epoch 24/200 - Loss: 0.33402562141418457
Epoch 25/200 - Loss: 0.17235764861106873
Epoch 26/

In [43]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

# Get the predicted labels for the test data
predicted_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        predicted_labels.extend(predicted.tolist())


# Convert the predicted labels back to original labels
predicted_labels = label_encoder.inverse_transform(predicted_labels)

# Convert the test labels to the same type as predicted labels
y_test = label_encoder.inverse_transform(y_test)

# Calculate precision, recall, F1-score, and accuracy
test_precision = precision_score(y_test, predicted_labels, average='weighted')
test_recall = recall_score(y_test, predicted_labels, average='weighted')
test_f1_score = f1_score(y_test, predicted_labels, average='weighted')
test_accuracy = accuracy_score(y_test, predicted_labels)

# Print the performance metrics
print("-------- Test Set Performance --------")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Test F1-Score: {test_f1_score}")
print(f"Test Accuracy: {test_accuracy}")



Accuracy: 40.625%
-------- Test Set Performance --------
Test Precision: 0.43543900995282425
Test Recall: 0.40625
Test F1-Score: 0.4052766859093274
Test Accuracy: 0.40625
